In [ ]:
import time
import csv
import re
import os

In [ ]:
def raw2cache():
    with open("url/raw", "r", encoding="utf-8") as f:
        data_raw = f.readlines()
        data_raw = [e.strip() for e in data_raw]
        data_dict = []
        for stt in range(1, 9999):
            idxs = [i for i, x in enumerate(data_raw) if x == str(stt)]
            if len(idxs) == 0:
                break
            idx = idxs[0]
            data_dict.append({
                "code": data_raw[idx+1],
                "name": data_raw[idx+2],
                "field": data_raw[idx+3],
                "org": data_raw[idx+4],
                "level": data_raw[idx+5],
            })
        data_dict = sorted(data_dict, key=lambda e: len(e["name"]))
    with open("url/cache", mode="w", newline="", encoding="utf-8") as f:
        fieldnames = data_dict[0].keys()
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data_dict)
raw2cache()

In [ ]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as Selenium_Service
from selenium.webdriver.chrome.options import Options as Selenium_Options
from selenium.webdriver import Chrome as Selenium_Chrome
chrome_options = Selenium_Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
selenium_service = Selenium_Service(ChromeDriverManager().install())
selenium_driver = Selenium_Chrome(service=selenium_service, options=chrome_options) # Start WebDriver
# sele_driver.quit() # Close the browser

In [ ]:
def scrape_html_content(url, waiting_time=1):
    selenium_driver.get(url)
    time.sleep(waiting_time)
    html_content = selenium_driver.page_source
    return html_content

scrape_patterns = {
    "Trình tự thực hiện": r""">Trình tự thực hiện</td>\s*<td[^>]*>(.*?)</td>""", 
    "Cách thức thực hiện": r""">Cách thức thực hiện</td>\s*<td[^>]*>(.*?)</td>""", 
    "Thành phần hồ sơ": r""">Thành phần hồ sơ</td>\s*<td[^>]*>(.*?)</td>""", 
    "Thời gian giải quyết": r""">Thời gian giải quyết</td>\s*<td[^>]*>(.*?)</td>""", 
    "Đối tượng thực hiện": r""">Đối tượng thực hiện</td>\s*<td[^>]*>(.*?)</td>""", 
    "Cơ quan thực hiện": r""">Cơ quan thực hiện</td>\s*<td[^>]*>(.*?)</td>""", 
    "Kết quả": r""">Kết quả</td>\s*<td[^>]*>(.*?)</td>""", 
    "Phí, lệ phí": r""">Phí, lệ phí</td>\s*<td[^>]*>(.*?)</td>""", 
    "Tên mẫu đơn, tờ khai": r""">Tên mẫu đơn, tờ khai</td>\s*<td[^>]*>(.*?)</td>""", 
    "Yêu cầu, điều kiện": r""">Yêu cầu, điều kiện</td>\s*<td[^>]*>(.*?)</td>""", 
    "Căn cứ pháp lý": r""">Căn cứ pháp lý</td>\s*<td[^>]*>(.*?)</td>""", 
}

with open("url/cache", mode="r", newline="", encoding="utf-8") as f:
    thutucs = [e for e in csv.DictReader(f)]
    tenthutucs = [e['name'] for e in thutucs]

cache2_machuans = []
if os.path.exists("url/cache_2"):
    with open("url/cache_2", mode="r", newline="", encoding="utf-8") as f:
        cache2 = [e for e in csv.DictReader(f)]
        cache2_machuans = [e['code'] for e in cache2]




for i, thutuc in enumerate(thutucs):
    machuan = thutuc["code"]
    final_res = thutuc

    skip = False
    if final_res["code"] not in cache2_machuans:
        url_1 = "https://dichvucong.lamdong.gov.vn/vi/procedure/search?keyword=" + machuan
        html_1 = scrape_html_content(url_1)
        match_1 = re.search(r'href="(/vi/procedure/detail/[a-zA-Z0-9]+)"', html_1)
        if match_1:
            url_2 = "https://dichvucong.lamdong.gov.vn" + match_1.group(1)
            html_2 = scrape_html_content(url_2)
            # ----------
            final_res["link"] = url_2
            # ----------
            match_3 = re.findall(r'<p[^>]*>(.*?)</p>', html_2, re.DOTALL)
            final_res["related"] = " | ".join([e for e in match_3 if e in tenthutucs])
            # ----------
            for i in range(len(scrape_patterns)):
                pattern_name = list(scrape_patterns.keys())[i]
                pattern_regex = list(scrape_patterns.values())[i]
                match_2 = re.search(pattern_regex, html_2, re.DOTALL)
                if match_2:
                    extracted_text = match_2.group(1)
                    extracted_text = re.sub(r'</?[^>]+>', '\n', extracted_text) # Replace HTML tags with "\n"
                    extracted_text = re.sub(r'\n+', '\n', extracted_text)       # Merge multiple "\n"
                    extracted_text = extracted_text.replace("&nbsp;", " ")      # Replace &nbsp; with " "
                    extracted_text = extracted_text.strip()
                    extracted_text = extracted_text if extracted_text != "" else f"Không có thông tin {pattern_name.lower()}."
                    final_res[pattern_name] = extracted_text
            # ----------
    else:
        skip = True
    if skip:
        print("⏩", end="")
    elif len(final_res) == 18:
        if not os.path.exists("url/cache_2"):
            with open("url/cache_2", mode="w", newline="", encoding="utf-8") as f:
                fieldnames = [final_res][0].keys()
                writer = csv.DictWriter(f, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows([final_res])
            with open("url/cache_2", mode="r", newline="", encoding="utf-8") as f:
                cache2 = [e for e in csv.DictReader(f)]
                cache2_machuans = [e['code'] for e in cache2]
        if final_res["code"] not in cache2_machuans:
            cache2.append(final_res)
            with open("url/cache_2", mode="w", newline="", encoding="utf-8") as f:
                fieldnames = cache2[0].keys()
                writer = csv.DictWriter(f, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(cache2)
        print("✔️", end="")
    else:
        print("❌", end="")
    if i % 50 == 49: print()